In [9]:
using Plots
using TensorCrossInterpolation, LinearAlgebra
using QuanticsTCI
using QuanticsGrids
using ITensorMPS
using ITensors
using NDTensors
using TCIITensorConversion
using IterativeSolvers
using LinearMaps
using LinearAlgebra
include("src/differential_mpo.jl")

get_c_vec (generic function with 2 methods)

In [10]:
# simulation parameter

R = 10
tol = 1e-8
max_bond = 10
delta_t = 0.1 * 2.0^-(R-1)
v = 1e-5
penalty_coefficient = 2.5e5


# starting conditions and boundaries

a = 2
xmin = -10
xmax = 10
ymin = -10
ymax = 10
dx = float(abs(xmin - xmax) / (2^R-1))
dy = float(abs(ymin - ymax) / (2^R-1))

function u_1(x,y)
    return exp(-(((x/a)^2/2) + ((y/a)^2)/2))
end
function u_2(x,y)
    return exp(-(((x/a)^2/2) + ((y/a)^2)/2))
end

u_2 (generic function with 1 method)

In [11]:
# QuanticsTCI
qgrid = DiscretizedGrid{2}(R, (xmin, ymin), (xmax, ymax); includeendpoint = true)
u1q, ranks1, errors1 = quanticscrossinterpolate(Float64, u_1, qgrid; maxbonddim=max_bond)
u2q, ranks2, errors2 = quanticscrossinterpolate(Float64, u_2, qgrid; maxbonddim=max_bond)

# Convert to MPS Format
s = siteinds("Qudit", R, dim=4)
u = [ITensorMPS.MPS(TensorTrain(u1q.tci), sites=s), ITensorMPS.MPS(TensorTrain(u2q.tci), sites=s)]
u_p = deepcopy.(u)

# Build Diff MPOs
d1 = [Diff_1_8_x(dx, s), Diff_1_8_y(dy, s)]
d2 = [Diff_2_8_x(dx, s), Diff_2_8_y(dy, s)]
del = MPO([delta(s[i], s[i]', s[i]'') for i in 1:length(s)])
print("Done!")

Done!

In [12]:
center = 2
u = orthogonalize.(u, center)
u_p = orthogonalize.(u_p, center)
c_vec = get_c_vec(u_p, center)
beta = make_beta(u, u_p, center, delta_t, v, d1, d2, del, max_bond)

function A_function(c_vec)
    return c_vec - penalty_coefficient * delta_t^2 * apply_H(c_vec, u_p, d1, center, max_bond)
end
d = sum(length.(array.(getindex.(u_p, center))))
A = FunctionMap{Float64,false}(A_function, d)

320×320 FunctionMap{Float64,false}(A_function; issymmetric=false, ishermitian=false, isposdef=false)

In [13]:
A_dense = Matrix(A)

320×320 Matrix{Float64}:
  1.23915      -0.0107038     7.38156e-17  …  -0.000555927   3.9125e-5
 -0.0107098     1.23915      -3.5474e-18       1.03514e-5   -0.000555927
  1.30685e-17   4.02211e-19   1.23915          3.74749e-19   4.91654e-19
 -1.11606e-17  -2.01851e-17  -0.0107098       -3.20253e-19   2.89904e-19
  0.0           2.5551e-44    5.24416e-36     -4.65513e-19  -7.9886e-7
  0.0           6.18509e-41  -3.54241e-36  …   1.17732e-6    2.201e-19
  0.0           2.70459e-59   2.02839e-32      6.39949e-20   0.000193217
  0.0           4.8184e-59    9.1585e-20      -3.7676e-5     7.1267e-20
  2.73736e-19   0.0107098     6.12898e-19     -2.48669e-19  -1.03514e-5
  0.0107038    -3.76739e-17  -4.14011e-19     -3.9125e-5    -1.23649e-19
  ⋮                                        ⋱                
 -0.000209794  -6.17918e-20   4.38236e-20      5.57688e-33  -3.47341e-19
  2.42469e-18  -3.3748e-5    -1.08695e-18     -4.38984e-19   1.06865e-35
 -0.000201682   1.24887e-18   0.000209794     

In [14]:
eigs = eigvals(A_dense)
display(eigs)

320-element Vector{ComplexF64}:
 0.9996887197886923 - 6.529751445816327e-5im
 0.9996887197886923 + 6.529751445816327e-5im
 0.9997220671060767 + 0.0im
 0.9998472403206422 - 5.9810385038805966e-5im
 0.9998472403206422 + 5.9810385038805966e-5im
     0.999861299291 + 0.0im
 0.9998802842168641 - 2.6594701104165202e-5im
 0.9998802842168641 + 2.6594701104165202e-5im
 0.9998924014984591 + 0.0im
 0.9999610716298872 - 7.584105606046372e-5im
                    ⋮
  4.050983527614778 + 4.045209243544021e-5im
  4.050987733389725 - 4.7523703300834256e-5im
  4.050987733389725 + 4.7523703300834256e-5im
   4.05100433430734 - 4.2950331767497244e-5im
   4.05100433430734 + 4.2950331767497244e-5im
  4.350962302549678 + 0.0im
   4.35097222130214 - 1.5778225857354982e-6im
   4.35097222130214 + 1.5778225857354982e-6im
  4.350983055625644 + 0.0im

In [15]:
println(c_vec)
# solve system
history = cg!(c_vec, A, beta, abstol=0, reltol=1e-4, verbose=false, maxiter=100, log=true)[2]
println("Center $center: $history")
println(c_vec)
norm(A(c_vec) - beta)

[-90.61732447537997, -1.4494017147313518, -1.4626209743543659, 0.04109387572864399, 0.013333803897092067, -0.047352898513139244, -0.05057385149976952, -12.8183556870415, 0.1683329499522633, 34.78500311329947, 0.0035374225442433697, 1.2737914822496628, 0.8086695892315443, -0.0025264223070578114, 34.87478940434427, 0.5085345537582268, 0.0, 0.15631083393800163, 0.15854601689174108, 0.010056128899261878, 0.019388229614132783, -1.5656396163809694, -1.474695231819084, -89.9042183099129, 0.47044434243199984, 30.063328285147307, -0.002137353840409319, 0.5839060260249319, 1.155926348762278, 0.0027256023466387556, 29.84793484467211, 0.07190376147795254, 0.0, -0.3861614417509491, -0.017886337665593156, -0.017926006707260796, -0.013869373556681502, -0.4442991701433294, -0.009767424625003431, 0.0, -1.4268718734110457, -78.06171398556377, -0.05912739135703204, -3.3018241696019937, 0.5551111255633725, 0.008428184515448287, 12.300665300127903, 0.20404892755496773, 0.0, -0.056664454405162434, -0.393836

0.16653072234776795